# Goodreads Data Analyst with Spark
## Introduccion
En este proyecto, se realizó un análisis de datos de libros obtenidos de la página web Goodreads. Para llevar a cabo este análisis, se utilizaron técnicas de web scraping, procesamiento y análisis de datos con pandas y PySpark.

### Webscraping
#### 1.Obtención de información de la página "Goodreads" sobre libros electrónicos.

El código importa la función "get" del módulo "requests" y establece la URL de la página web "Goodreads" que muestra una lista de libros electrónicos. Luego, se usa la función "get" para enviar una solicitud de obtención de información a la URL.

In [1]:
from requests import get
url = 'https://www.goodreads.com/shelf/show/ebooks'
response = get(url)
print(response.text[:500])

<!DOCTYPE html>
<html class="desktop withSiteHeaderTopFullImage
">
<head>
  <title>Ebooks Shelf</title>

<meta content='Books shelved as ebooks: The Hunger Games by Suzanne Collins, Catching Fire by Suzanne Collins, Mockingjay by Suzanne Collins, Divergent by Veronica Roth...' name='description'>
<meta content='telephone=no' name='format-detection'>
<link href='https://www.goodreads.com/shelf/show/ebooks' rel='canonical'>



    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date()


El código imprime los primeros 500 caracteres del contenido HTML de la página web solicitada. Esto puede incluir el código fuente de la página, así como cualquier texto visible que se muestre en la página. En este caso, es probable que la salida incluya información sobre los libros electrónicos que se muestran en la página de "Goodreads".

#### 2.Análisis de HTML con BeautifulSoup.
El código importa la clase "BeautifulSoup" del módulo "bs4" y crea una instancia de esta clase utilizando el contenido HTML de la respuesta obtenida en la solicitud anterior. La cadena de texto HTML se pasa como el primer argumento y 'html.parser' se utiliza como el segundo argumento para indicar el analizador HTML que debe ser utilizado para analizar el contenido HTML.


In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

El código imprime el tipo de objeto creado por BeautifulSoup a partir del contenido HTML analizado. Esto puede ser útil para asegurarse de que el objeto creado es un objeto BeautifulSoup y que se puede manipular de manera efectiva para extraer información útil del contenido HTML. En este caso, se espera que la salida sea 

```
<class 'bs4.BeautifulSoup'>
```
lo que indica que se ha creado con éxito un objeto BeautifulSoup a partir del contenido HTML.

#### 3.Extracción de datos específicos de una página web utilizando BeautifulSoup.

El código utiliza el objeto BeautifulSoup creado en el código anterior para buscar todos los contenedores de libros en la página web utilizando el método "find_all". Se busca todos los elementos "div" que tienen una clase "elementList".

In [3]:
book_containers = html_soup.find_all('div', class_ ="elementList")
print(type(book_containers))
print(len(book_containers))

<class 'bs4.element.ResultSet'>
52


El código imprime el tipo de objeto devuelto por la función "find_all" y la cantidad de elementos encontrados. En este caso, se espera que la salida sea
```
# <class 'bs4.element.ResultSet'>
```
y un número entero que indica la cantidad de contenedores de libros encontrados en la página web. Esto indica que se han encontrado con éxito todos los contenedores de libros en la página y se han almacenado en una lista.

#### 4.Extracción de información detallada de contenedores de libros en una página web utilizando BeautifulSoup.
El código crea varias listas vacías para almacenar la información detallada de cada libro, como el nombre, el autor, la calificación y el año de publicación. Luego, utiliza un bucle "for" para iterar a través de cada contenedor de libros en la lista "book_containers" que se creó anteriormente.

In [4]:
names = []
years = []
ratings = []
avgscores = []
author = []

max_books = 50
book_count = 0

for container in book_containers:
    if container.find('div', class_ = 'left') is not None:
        name = container.find('a',class_="bookTitle").text
        names.append(name)
        authors = container.find('a',class_="authorName").text
        author.append(authors)
        votes = container.find('span',class_="greyText smallText").text
        votes = votes.split()
        scores = votes[2]
        avgscores.append(scores)
        rates = votes[4]
        ratings.append(rates)
        year = votes[8] 
        years.append(year)
        
        book_count += 1
        if book_count >= max_books:
            break

Dentro del bucle "for", el código verifica si el contenedor de libros contiene un elemento "div" con clase "left". Si lo hace, extrae el nombre del libro, el autor, las calificaciones, el promedio de puntuaciones y el año de publicación utilizando los métodos "find" y "text" de BeautifulSoup. Luego, agrega cada pieza de información a su respectiva lista.

Además, el código tiene un límite de libros "max_books" que se establece en 50 y una variable "book_count" que se utiliza para contar el número de libros que se han agregado a las listas. Si "book_count" alcanza "max_books", el bucle "for" se detiene utilizando la sentencia "break". La salida del código no imprime nada directamente, pero las listas creadas se pueden utilizar para obtener información detallada sobre los libros encontrados en la página web.

#### 5.Creación de un DataFrame de Pandas para almacenar información detallada de los libros.
El código importa el módulo Pandas y utiliza la función "DataFrame" de Pandas para crear un nuevo DataFrame llamado "test_df". El DataFrame se crea utilizando un diccionario que contiene las listas de información detallada de los libros que se crearon anteriormente.

In [5]:
import pandas as pd
test_df = pd.DataFrame({'booktitle': names,
'published': years,
'ratings': ratings,
'avg_score': avgscores,
'author': author
})
test_df

,booktitle,published,ratings,avg_score,author
0,"The Hunger Games (The Hunger Games, #1)",2008,"7,909,864",4.33,Suzanne Collins
1,"Catching Fire (The Hunger Games, #2)",2009,"3,207,102",4.31,Suzanne Collins
2,"Mockingjay (The Hunger Games, #3)",2010,"2,921,971",4.07,Suzanne Collins
3,"Divergent (Divergent, #1)",2011,"3,685,950",4.15,Veronica Roth
4,Pride and Prejudice (Paperback),1813,"3,916,631",4.28,Jane Austen
5,The Fault in Our Stars (Hardcover),2012,"4,732,191",4.15,John Green
6,Harry Potter and the Philosopher’s Stone (Harr...,1997,"9,220,226",4.47,J.K. Rowling
7,"Insurgent (Divergent, #2)",2012,"1,410,052",3.98,Veronica Roth
8,"Fifty Shades of Grey (Fifty Shades, #1)",2011,"2,421,179",3.66,E.L. James
9,"A Game of Thrones (A Song of Ice and Fire, #1)",1996,"2,345,009",4.44,George R.R. Martin


El código imprime el contenido del DataFrame "test_df" creado. El DataFrame tiene cinco columnas ("booktitle", "published", "ratings", "avg_score", y "author") y filas que representan cada libro encontrado en la página web. Cada fila contiene la información detallada del libro correspondiente en cada columna. Este DataFrame puede ser utilizado para analizar y visualizar la información detallada de los libros encontrados en la página web.

### Procesamiento de datos
#### 1.Verificación de tipos de datos en el DataFrame de Pandas.
El código llama al método "dtypes" en el DataFrame "test_df" creado anteriormente para obtener información sobre los tipos de datos de cada columna en el DataFrame.

Es importante verificar los tipos de datos de cada columna en un DataFrame de Pandas para asegurarse de que los datos sean coherentes y se puedan analizar correctamente. Si las columnas tienen tipos de datos incorrectos, esto puede llevar a errores en el análisis y en la interpretación de los resultados.

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   booktitle  50 non-null     object
 1   published  50 non-null     object
 2   ratings    50 non-null     object
 3   avg_score  50 non-null     object
 4   author     50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


El código imprime los tipos de datos de cada columna del DataFrame "test_df" utilizando el método "dtypes".

#### 2.Conversión de tipos de datos en el DataFrame de Pandas.
El código utiliza los métodos "astype" y "str.replace" en el DataFrame "test_df" para cambiar los tipos de datos de las columnas "published", "ratings" y "avg_score". 

En particular, la columna "published" se convierte a un tipo de datos entero ("int"), la columna "ratings" se convierte a un tipo de datos entero y la columna "avg_score" se convierte a un tipo de datos de punto flotante ("float").

In [7]:
test_df['published'] = test_df['published'].astype(int)
test_df['ratings'] = test_df['ratings'].str.replace(',', '').astype(int)
test_df['avg_score'] = test_df['avg_score'].astype(float)

test_df.dtypes

booktitle     object
published      int64
ratings        int64
avg_score    float64
author        object
dtype: object

El código imprime los tipos de datos de cada columna en el DataFrame "test_df" después de que se han realizado los cambios de tipo de datos. La columna "published" se ha cambiado correctamente a un tipo de datos entero, mientras que las columnas "ratings" y "avg_score" se han cambiado correctamente a sus tipos de datos.

Sin embargo, las columnas "author" y "booktitle" no se han cambiado de tipo de datos y siguen siendo "object". Esto se debe a que las columnas contienen valores de texto ("strings") que son específicos para cada libro. 

Por lo tanto el tipo de datos "object" es apropiado para almacenar los valores de texto en estas columnas.

#### 3.Instalación y configuración de PySpark.
El código instala y configura PySpark en el entorno de desarrollo. Primero, instala Java Development Kit 8 (JDK 8) utilizando el comando "apt-get" y silencia la salida utilizando "> /dev/null". Luego, descarga el archivo tar de Spark 3.1.2 utilizando el comando "wget" y extrae el archivo tar utilizando el comando "tar". A continuación, instala el paquete "findspark" y el paquete "pyspark" utilizando el comando "pip".

In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=c30357f7a5dcb212401d0ae412671ec015898bb55cf48d4cd46830a187b58a2a
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


#### 3.Configuración de variables de entorno para PySpark.
El código configura dos variables de entorno en el entorno de desarrollo utilizando el módulo "os". La primera variable de entorno "JAVA_HOME" se establece en el directorio donde se encuentra instalado el JDK 8. La segunda variable de entorno "SPARK_HOME" se establece en el directorio donde se ha extraído el archivo tar de Spark 3.1.2.

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

#### 4.Inicialización de SparkSession y configuración de aplicación.
El código utiliza el módulo "findspark" para inicializar Spark en el entorno de desarrollo y luego importa la clase "SparkSession" y la función "col" y "avg" del módulo "pyspark.sql". Luego, crea una instancia de la clase "SparkSession" utilizando el método "builder" y configura el nombre de la aplicación como "App-Goodreads".

In [10]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

spark = SparkSession.builder \
    .appName("App-Goodreads") \
    .getOrCreate()

### 5.Conversión del DataFrame de Pandas a DataFrame de PySpark y definición del esquema.
El código importa la clase "StructType", "StructField", "StringType", "IntegerType" y "FloatType" del módulo "pyspark.sql.types". Luego, define un esquema utilizando la clase "StructType" y la clase "StructField" que define el nombre y el tipo de datos para cada columna en el DataFrame de PySpark. A continuación, convierte el DataFrame de Pandas "test_df" en un DataFrame de PySpark utilizando el método "createDataFrame" de la clase "SparkSession".

In [11]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([
    StructField("booktitle", StringType(), True),
    StructField("published", IntegerType(), True),
    StructField("ratings", IntegerType(), True),
    StructField("avg_score", FloatType(), True),
    StructField("author", StringType(), True)
])

test_spark_df = spark.createDataFrame(test_df, schema=schema)
test_spark_df.printSchema()

root
 |-- booktitle: string (nullable = true)
 |-- published: integer (nullable = true)
 |-- ratings: integer (nullable = true)
 |-- avg_score: float (nullable = true)
 |-- author: string (nullable = true)



El DataFrame de PySpark se convierte porque algunos algoritmos o herramientas de análisis de datos solo están disponibles en PySpark y no en pandas. En este caso, el DataFrame original "test_df" se crea utilizando pandas y se utiliza para recopilar y almacenar información detallada de los libros de una página web. La conversión se realiza porque PySpark proporciona una interfaz para trabajar con conjuntos de datos distribuidos y puede manejar grandes conjuntos de datos con eficacia.

Una vez que el DataFrame "test_df" se convierte en un DataFrame de PySpark, se pueden utilizar diferentes herramientas y algoritmos de análisis de datos en Spark para procesar y analizar los datos de forma eficiente. Al convertir el DataFrame de pandas a un DataFrame de PySpark, los usuarios pueden aprovechar las características y capacidades avanzadas de PySpark, como el procesamiento distribuido, el procesamiento en tiempo real, la tolerancia a fallos y la escalabilidad horizontal. En general, la conversión de un DataFrame de pandas a un DataFrame de PySpark permite a los usuarios trabajar con grandes conjuntos de datos y procesarlos de manera eficiente con herramientas de análisis de datos avanzadas.

#### 6.Registro del DataFrame de PySpark como tabla temporal.
El código utiliza el método "createOrReplaceTempView" en el DataFrame de PySpark "test_spark_df" para registrarlo como una tabla temporal. El nombre de la tabla temporal se establece como "books".

In [12]:
# Registrar el DataFrame como una tabla temporal
test_spark_df.createOrReplaceTempView("books")

Registramos el DataFrame de PySpark "test_spark_df" como una tabla temporal utilizando el método "createOrReplaceTempView". Al registrar el DataFrame de PySpark como una tabla temporal, los usuarios pueden realizar consultas SQL en el DataFrame utilizando la sintaxis SQL, lo que facilita el análisis de los datos. Por lo tanto, los usuarios pueden utilizar la tabla temporal "books" en las consultas SQL posteriores para analizar y procesar los datos de forma más eficiente.

####7. Testeo Spark
Una vez que el DataFrame de PySpark se ha registrado como una tabla temporal, los usuarios pueden utilizar la sintaxis SQL para realizar consultas en el DataFrame. Esto es particularmente útil para analizar grandes conjuntos de datos, ya que la sintaxis SQL es más fácil de entender y utilizar que la sintaxis de Python para el procesamiento de datos.

**7.1 Filtrar libros con una calificación promedio superior a un cierto umbral**

In [13]:
high_rated_books = spark.sql("SELECT * FROM books WHERE avg_score > 4.0")
high_rated_books.show(truncate=False)

+-------------------------------------------------------------+---------+-------+---------+------------------+
|booktitle                                                    |published|ratings|avg_score|author            |
+-------------------------------------------------------------+---------+-------+---------+------------------+
|The Hunger Games (The Hunger Games, #1)                      |2008     |7909864|4.33     |Suzanne Collins   |
|Catching Fire (The Hunger Games, #2)                         |2009     |3207102|4.31     |Suzanne Collins   |
|Mockingjay (The Hunger Games, #3)                            |2010     |2921971|4.07     |Suzanne Collins   |
|Divergent (Divergent, #1)                                    |2011     |3685950|4.15     |Veronica Roth     |
|Pride and Prejudice (Paperback)                              |1813     |3916631|4.28     |Jane Austen       |
|The Fault in Our Stars (Hardcover)                           |2012     |4732191|4.15     |John Green        |
|

**7.2 Obtener el promedio de las calificaciones de los libros publicados en un rango de años específico**

In [14]:
average_ratings = spark.sql("SELECT AVG(avg_score) as avg_ratings FROM books WHERE published BETWEEN 2000 AND 2010")
average_ratings.show()

+-----------------+
|      avg_ratings|
+-----------------+
|4.372999954223633|
+-----------------+



**7.3 Encontrar los autores que tienen más de un libro en la lista**

In [15]:
multiple_books_authors = test_spark_df.groupBy("author").count().filter(col("count") > 1)
multiple_books_authors.show()

+------------------+-----+
|            author|count|
+------------------+-----+
|George R.R. Martin|    2|
|   Suzanne Collins|    3|
|        E.L. James|    2|
|     Sarah J. Maas|    2|
|     Veronica Roth|    3|
|      J.K. Rowling|    7|
+------------------+-----+



### Analisis de los datos
#### 1.Libros con la mejor calificación promedio por año.
**PySpark**

In [16]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

windowSpec = Window.partitionBy("published").orderBy(desc("avg_score"))

best_books_by_year_sql = test_spark_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") == 1) \
    .drop("row_number") \
    .select("published", "booktitle", "author", "avg_score") \
    .orderBy("published")

best_books_by_year_sql.show(truncate=False)

+---------+------------------------------------------------------------+---------------------------+---------+
|published|booktitle                                                   |author                     |avg_score|
+---------+------------------------------------------------------------+---------------------------+---------+
|1813     |Pride and Prejudice (Paperback)                             |Jane Austen                |4.28     |
|1818     |Frankenstein: The 1818 Text (Paperback)                     |Mary Wollstonecraft Shelley|3.85     |
|1847     |Jane Eyre (Paperback)                                       |Charlotte Brontë           |4.14     |
|1868     |Little Women (Paperback)                                    |Louisa May Alcott          |4.14     |
|1890     |The Picture of Dorian Gray (Paperback)                      |Oscar Wilde                |4.12     |
|1897     |Dracula (Paperback)                                         |Bram Stoker                |4.01     |
|

**Pandas**

In [17]:
best_books_by_year_pd = test_df.groupby('published').agg({'booktitle': 'first', 'author': 'first', 'avg_score': 'max'}).reset_index().sort_values('published')
best_books_by_year_pd

,published,booktitle,author,avg_score
0,1813,Pride and Prejudice (Paperback),Jane Austen,4.28
1,1818,Frankenstein: The 1818 Text (Paperback),Mary Wollstonecraft Shelley,3.85
2,1847,Jane Eyre (Paperback),Charlotte Brontë,4.14
3,1868,Little Women (Paperback),Louisa May Alcott,4.14
4,1890,The Picture of Dorian Gray (Paperback),Oscar Wilde,4.12
5,1897,Dracula (Paperback),Bram Stoker,4.01
6,1925,The Great Gatsby (Paperback),F. Scott Fitzgerald,3.93
7,1949,1984 (Paperback),George Orwell,4.19
8,1985,"The Handmaid’s Tale (The Handmaid's Tale, #1)",Margaret Atwood,4.13
9,1996,"A Game of Thrones (A Song of Ice and Fire, #1)",George R.R. Martin,4.44


Como podemos ver el objetivo es proporcionar una lista de los mejores libros de cada año, lo que puede ser útil para los usuarios que buscan recomendaciones de libros de alta calidad para leer.

#### 2.Cantidad de libros publicados por década.
**PySpark**

In [18]:
from pyspark.sql.functions import floor

# Calcular la década y realizar la consulta en PySpark
books_by_decade = test_spark_df \
    .withColumn("decade", (floor(col("published") / 10) * 10)) \
    .groupBy("decade") \
    .count() \
    .orderBy("decade")

books_by_decade.show()

+------+-----+
|decade|count|
+------+-----+
|  1810|    2|
|  1840|    1|
|  1860|    1|
|  1890|    2|
|  1920|    1|
|  1940|    1|
|  1980|    1|
|  1990|    6|
|  2000|    9|
|  2010|   25|
|  2020|    1|
+------+-----+



**Pandas**

In [19]:
test_df['decade'] = (test_df['published'] // 10) * 10
books_by_decade_pd = test_df.groupby('decade').size().reset_index(name='num_books').sort_values('decade')
books_by_decade_pd

,decade,num_books
0,1810,2
1,1840,1
2,1860,1
3,1890,2
4,1920,1
5,1940,1
6,1980,1
7,1990,6
8,2000,9
9,2010,25


Como podemos analizar la cantidad de libros publicados por década para proporcionar información sobre cómo ha evolucionado esta cantidad a lo largo del tiempo. El análisis de la cantidad de libros publicados por década puede proporcionar información valiosa sobre la evolución de la industria editorial y las tendencias literarias.

#### 3.Autores con la calificación promedio más alta (considerando solo autores con al menos N libros, por ejemplo, N = 3).
**PySpark**

In [20]:
from pyspark.sql.functions import round, count
N = 3
top_authors = test_spark_df.groupBy("author") \
    .agg(round(avg("avg_score"), 2).alias("avg_rating"),
         count("booktitle").alias("num_books")) \
    .filter(col("num_books") >= N) \
    .orderBy(desc("avg_rating"))

top_authors.show()

+---------------+----------+---------+
|         author|avg_rating|num_books|
+---------------+----------+---------+
|   J.K. Rowling|      4.53|        7|
|Suzanne Collins|      4.24|        3|
|  Veronica Roth|      3.91|        3|
+---------------+----------+---------+



**Pandas**

In [21]:
N = 3
author_group = test_df.groupby('author').agg({'avg_score': 'mean', 'booktitle': 'count'}).rename(columns={'avg_score': 'avg_rating', 'booktitle': 'num_books'})
author_group['avg_rating'] = author_group['avg_rating'].round(2)
top_authors_pd = author_group[author_group['num_books'] >= N].sort_values('avg_rating', ascending=False)

top_authors_pd

,avg_rating,num_books
author,,
J.K. Rowling,4.53,7
Suzanne Collins,4.24,3
Veronica Roth,3.91,3


Lo que se busca identificar es a los autores con la calificación promedio más alta en base a los libros que han publicado, considerando solo aquellos autores que hayan publicado al menos N libros (donde N es un número determinado, por ejemplo, N = 3). Este análisis puede ser útil para los usuarios que buscan descubrir nuevos autores o que desean conocer mejor el trabajo de autores existentes.

#### 4.Libros con una calificación promedio superior a la calificación promedio general.
**PySpark**

In [22]:
# PySpark
overall_avg_rating = test_spark_df.agg(avg(col("avg_score"))).collect()[0][0]
books_above_avg = test_spark_df.filter(col("avg_score") > overall_avg_rating)
books_above_avg.show(truncate=False)

+-------------------------------------------------------------+---------+-------+---------+-------------------+
|booktitle                                                    |published|ratings|avg_score|author             |
+-------------------------------------------------------------+---------+-------+---------+-------------------+
|The Hunger Games (The Hunger Games, #1)                      |2008     |7909864|4.33     |Suzanne Collins    |
|Catching Fire (The Hunger Games, #2)                         |2009     |3207102|4.31     |Suzanne Collins    |
|Pride and Prejudice (Paperback)                              |1813     |3916631|4.28     |Jane Austen        |
|Harry Potter and the Philosopher’s Stone (Harry Potter, #1)  |1997     |9220226|4.47     |J.K. Rowling       |
|A Game of Thrones (A Song of Ice and Fire, #1)               |1996     |2345009|4.44     |George R.R. Martin |
|A Court of Thorns and Roses (A Court of Thorns and Roses, #1)|2015     |1431452|4.2      |Sarah J. Maas

**Pandas**

In [23]:
overall_avg_rating_pd = test_df['avg_score'].mean()
books_above_avg_pd = test_df[test_df['avg_score'] > overall_avg_rating_pd]
books_above_avg_pd

,booktitle,published,ratings,avg_score,author,decade
0,"The Hunger Games (The Hunger Games, #1)",2008,7909864,4.33,Suzanne Collins,2000
1,"Catching Fire (The Hunger Games, #2)",2009,3207102,4.31,Suzanne Collins,2000
4,Pride and Prejudice (Paperback),1813,3916631,4.28,Jane Austen,1810
6,Harry Potter and the Philosopher’s Stone (Harr...,1997,9220226,4.47,J.K. Rowling,1990
9,"A Game of Thrones (A Song of Ice and Fire, #1)",1996,2345009,4.44,George R.R. Martin,1990
12,A Court of Thorns and Roses (A Court of Thorns...,2015,1431452,4.20,Sarah J. Maas,2010
15,Harry Potter and the Chamber of Secrets (Harry...,1998,3571416,4.43,J.K. Rowling,1990
20,Harry Potter and the Prisoner of Azkaban (Harr...,1999,3777941,4.58,J.K. Rowling,1990
22,"The Deal (Off-Campus, #1)",2015,528636,4.26,Elle Kennedy,2010
23,The Martian (Hardcover),2011,1030896,4.41,Andy Weir,2010


Lo que se busca es identificar los libros que tienen una calificación promedio superior a la calificación promedio general de todos los libros. Este análisis puede ser útil para los usuarios que buscan descubrir nuevos libros de alta calidad para leer.

#### 5.Detener la sesión de PySpark.
Para terminar esta seccion debemos detener la sesión de PySpark se liberan los recursos utilizados por la sesión actual y se cierra la conexión con cualquier sistema de almacenamiento de datos que se haya utilizado en la sesión actual.

In [24]:
spark.stop()

## Conclusion
Durante este proyecto, hemos trabajado en un proyecto de análisis de datos utilizando PySpark y técnicas de web scraping. Hemos recopilado datos de libros de una plataforma en línea utilizando web scraping y hemos utilizado PySpark para analizar y visualizar estos datos.

En particular, hemos explorado cómo podemos analizar la cantidad de libros publicados por década, identificar a los autores con la calificación promedio más alta, identificar los libros con una calificación promedio superior a la calificación promedio general, y analizar los libros con la mejor calificación promedio por año.

Este proyecto nos ha permitido aprender cómo se puede utilizar PySpark para trabajar con grandes conjuntos de datos y cómo se pueden aplicar técnicas de web scraping para recopilar datos de fuentes en línea. También hemos aprendido cómo analizar y visualizar datos utilizando PySpark y las funciones integradas que proporciona.

Por lo tanto, este proyecto nos ha permitido desarrollar habilidades valiosas en el análisis de datos y nos ha enseñado cómo podemos aplicar estas habilidades en proyectos prácticos. Además, hemos podido explorar herramientas poderosas como PySpark y el web scraping, lo que nos permitirá aplicar estas habilidades en futuros proyectos y en el campo profesional.